In [1]:
import os
import torch
from torchvision import datasets, transforms
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data

In [2]:

data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

path = 'dataset/'
dataset = torchvision.datasets.ImageFolder(root=path, transform=data_transforms)

# Split the dataset into train and test
train_size = int(0.8 * len(dataset) )
test_size = len(dataset) - train_size
train_dataset, test_dataset = data.random_split(dataset, [train_size, test_size],generator=torch.Generator().manual_seed(42))

# Define the dataloaders
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=True)

trainloader=train_loader
testloader=test_loader

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader

In [4]:
# Define ResNet model with modified first layer
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.cuda()
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/home/mnabail/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/mnabail/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:

# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 10 == 9:
            print('[Epoch %d, Batch %d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0

[Epoch 1, Batch 10] loss: 0.686
[Epoch 1, Batch 20] loss: 0.517
[Epoch 2, Batch 10] loss: 0.321
[Epoch 2, Batch 20] loss: 0.270
[Epoch 3, Batch 10] loss: 0.214
[Epoch 3, Batch 20] loss: 0.192
[Epoch 4, Batch 10] loss: 0.166
[Epoch 4, Batch 20] loss: 0.145
[Epoch 5, Batch 10] loss: 0.116
[Epoch 5, Batch 20] loss: 0.104
[Epoch 6, Batch 10] loss: 0.153
[Epoch 6, Batch 20] loss: 0.096
[Epoch 7, Batch 10] loss: 0.169
[Epoch 7, Batch 20] loss: 0.066
[Epoch 8, Batch 10] loss: 0.121
[Epoch 8, Batch 20] loss: 0.106
[Epoch 9, Batch 10] loss: 0.084
[Epoch 9, Batch 20] loss: 0.047
[Epoch 10, Batch 10] loss: 0.076
[Epoch 10, Batch 20] loss: 0.064
[Epoch 11, Batch 10] loss: 0.043
[Epoch 11, Batch 20] loss: 0.042
[Epoch 12, Batch 10] loss: 0.047
[Epoch 12, Batch 20] loss: 0.016
[Epoch 13, Batch 10] loss: 0.042
[Epoch 13, Batch 20] loss: 0.083
[Epoch 14, Batch 10] loss: 0.016
[Epoch 14, Batch 20] loss: 0.023
[Epoch 15, Batch 10] loss: 0.050
[Epoch 15, Batch 20] loss: 0.023


In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('F1 Score: {:.2f}'.format(f1))

Accuracy: 94.59%
F1 Score: 0.94


In [7]:
# save model
torch.save(model.state_dict(), 'resnet18.pth')